In [1]:

import pickle

from GAMERNet.rnet.networks.reaction_network_paths import ReactionNetwork


with open('../scripts/C1_Cu111/rxn_net.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)  # dict of elementary reactions

rxn_net = ReactionNetwork().from_dict(content)
print(rxn_net)

ReactionNetwork(57 surface intermediates, 14 gas molecules, 105 elementary reactions)
Surface: Cu48(100)
Network Carbon cutoff: C1



In [2]:
source = '10111g'
target = '14111g'
n_inter_steps = 8

rxn_net.find_all_paths_from_sources_to_targets([source], [target], n_inter_steps)

{('10111g',
  '14111g'): [['10111g',
   '10111*',
   '10011*',
   '11011*',
   '12011*',
   '13011*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '10011*',
   '11011*',
   '12011*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '10011*',
   '11011*',
   '12111*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '10011*',
   '11111*',
   '12111*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '11211*',
   '11111*',
   '12111*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '11211*',
   '12211*',
   '12111*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '11211*',
   '12211*',
   '13211*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '11111*',
   '12111*',
   '13111*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '11112*',
   '11011*',
   '12011*',
   '13011*',
   '14111*',
   '14111g'], ['10111g',
   '10111*',
   '11112*',
   '11011*',
   '12011*

In [5]:
source_inter = '10111g'
target_inter = '14111g'
intermediate = '13011*'
n_inter_steps_inter = 12

rxn_net.find_paths_through_intermediate(source_inter, target_inter, intermediate, n_inter_steps_inter)

[['10111g',
  '10111*',
  '10011*',
  '11011*',
  '12011*',
  '13011*',
  '12011*',
  '13111*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '10011*',
  '11011*',
  '12011*',
  '13011*',
  '12011*',
  '12112*',
  '13111*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '10011*',
  '11011*',
  '12011*',
  '13011*',
  '12011*',
  '12112*',
  '13112*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '10011*',
  '11011*',
  '12011*',
  '13011*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '10011*',
  '11011*',
  '12011*',
  '13011*',
  '13112*',
  '12112*',
  '13111*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '10011*',
  '11011*',
  '12011*',
  '13011*',
  '13112*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '11112*',
  '11011*',
  '12011*',
  '13011*',
  '12011*',
  '13111*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111*',
  '11112*',
  '11011*',
  '12011*',
  '13011*',
  '12011*',
  '12112*',
  '13111*',
  '14111*',
  '14111g'],
 ['10111g',
  '10111